In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# đọc các file csv lên
books = pd.read_csv('../input/databooks/dataset/books.csv')
ratings = pd.read_csv('../input/databooks/dataset/ratings.csv')
book_tags = pd.read_csv('../input/databooks/dataset/book_tags.csv')
tags = pd.read_csv('../input/databooks/dataset/tags.csv')

In [ ]:
books['authors'] = books['authors'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x.split(', ')])

In [ ]:
def get_genres(x):
    t = book_tags[book_tags.goodreads_book_id==x]
    return [i.lower().replace(" ", "") for i in tags.tag_name.loc[t.tag_id].values]

In [ ]:
books['genres'] = books.book_id.apply(get_genres)

In [ ]:
books['soup'] = books.apply(lambda x: ' '.join([x['title']] + x['authors'] + x['genres']), axis=1)

In [ ]:
books.soup.head()

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity

count = CountVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
count_matrix = count.fit_transform(books['soup'])

In [ ]:
cosine_sim = cosine_similarity(count_matrix, count_matrix)

In [ ]:
indices = pd.Series(books.index, index=books['title'])
titles = books['title']

In [ ]:
def get_recommendations(title, n=10):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:31]
    book_indices = [i[0] for i in sim_scores]
    return list(titles.iloc[book_indices].values)[:n]

get_recommendations("The One Minute Manager")

# Collaborative Filtering 

In [ ]:
from surprise import Reader, Dataset, SVD
from surprise.model_selection import cross_validate

In [ ]:
reader = Reader()
data = Dataset.load_from_df(ratings[['user_id', 'book_id', 'rating']], reader)

In [ ]:
#Phương pháp suy biến SVD sẽ tìm ra một lớp các ma trận xấp xỉ tốt nhất với một ma trận cho trước, để giảm thiểu RMSE (Root Mean Square Error) và đưa ra các khuyến nghị tốt. 
svd = SVD()
cross_validate(svd, data, measures=['RMSE', 'MAE'])

In [ ]:
trainset = data.build_full_trainset()
svd.fit(trainset);

In [ ]:
ratings[ratings['user_id'] == 10]

In [ ]:
svd.predict(10,1506)

# Hybrid

In [ ]:
def hybrid(user_id, title, n=10):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:51]
    book_indices = [i[0] for i in sim_scores]
    
    df = books.iloc[book_indices][['book_id', 'title', 'original_publication_year', 'ratings_count']] # 'average_rating'
    df['new_rating'] = df['book_id'].apply(lambda x: svd.predict(user_id, x).est)
    df = df.sort_values('new_rating', ascending=False)
    return df.head(n)

In [ ]:
hybrid(4, 'Eat, Pray, Love')

In [ ]:
hybrid(10, 'Eat, Pray, Love')